In [24]:
import numpy as np
# import units as u
import numpy as np
import pandas as pd

import rasterio
from rasterio.plot import show_hist, show
import geopandas as gpd
import rasterstats
from shapely.geometry import shape

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

from icecream import ic

In [ ]:
def print_nice(float):
    return print(f"{float:,.0f}")

## calculate areas of states
- will take the average of everything on a state by state basis

In [2]:
# shape file definining administrative borders 
states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
states = gpd.read_file(states_path)

In [25]:
states_m = states.to_crs('epsg:32632')
state_areas = pd.DataFrame(states_m.apply(lambda x: (x.admin1Name, shape(x["geometry"]).area ), axis=1).to_list(), columns=["state", "area (m2)"])
state_areas.head()

# TODO => save this in csv or pickle file..

# state_areas["area (m2)"].sum()/ 1e6 # ~ 923,768 km2

,state,area (m2)
0,Abia,4.858882e+09
1,Adamawa,3.792499e+10
2,Akwa Ibom,6.736774e+09
3,Anambra,4.807933e+09
4,Bauchi,4.849640e+10


## calculate renewable resource 

In [6]:
# raster data sources => wind, temperature, solar flux 

data_paths = {
    "wind_speed": "data/Nigeria_MeanWindSpeed/NGA_wind-speed_100m.tif",
    "temperature": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/TEMP.tif",
    "solar_flux": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/DNI.tif",
}

In [8]:
def get_state_means(data_path, states=states):
    # read raster files..
    data = rasterio.open(data_path)
    data_array = data.read()[0]
    affine = data.transform

    # calculate zonal statistics
    averaged_data = rasterstats.zonal_stats(states, data_array, affine=affine, stats = ["mean"], geojson_out=True)

    state_wind_average = []
    for item in averaged_data:
        state_wind_average.append((item["properties"]["admin1Name"],
        item["properties"]["mean"]))

    df = pd.DataFrame(state_wind_average, columns=["state", "value"])
    return df


In [11]:
data_averages = {}
for name, path in data_paths.items():
    data_averages[name] = get_state_means(path, states=states)

# TODO => save this in csv or pickle file..
# TODO => need different wind speeds => need speeds at ground level for solar, and at 100 - 150 m for turbines 

In [19]:
avg_vals = {n: i["value"] for n, i in data_averages.items()}

## calculate solar resource

In [13]:
def calculate_power_panel(F_cur, T_a, w):
    """
    F_cur = _ # W/m^2 current solar flux normal to the panel 
    T_a = _ # K / ºC, air temperature the panel is exposed to 
    w = _ # m/s, wind speed panel is exposed to 
    => all of these are arrays 

    return power potential of a single panel in the area defined by the characteristics above
    
    """

    # values that change based on design! 
    D_f = 0.864 # derating factor, product of correction factors for additional processes affecting solar output, Table 5.2 
    E_panel = 0.18 # solar panel efficiency obtained under standard test conditions -- Ex 5.2
    A_panel = 1.5 # m^2, surface area of the pane -- Ex 5.2
    
    # 5.9, cell temperature, empirical so units do not eqate 
    T_c = T_a + 0.32 * (F_cur/(8.91 + 2*w))

    b_ref = 0.0025 # / K, temperature coefficient 
    T_th = 55 # K, threshold temeprature 
    T_ref = 298.15 # K, reference temperature 
    # 5.8, correction for cell temperature 
    C_temp = 1 - b_ref * np.maximum( np.minimum(T_c - T_ref, T_th ), 0 )

    # 5.7 actual AC power output from a solar panel at a given time 
    # P_ac = p_mpp_stc * C_temp * D_f / F_1000 # W based on panel rating
    P_ac = F_cur * A_panel * E_panel * C_temp * D_f # W, based on real conditions 

    return P_ac


def calc_power_potential(A_spacing, single_panel_potential):
    """
    given the area of a state, and the power potential of a panel in that state, calculate the power potential of the state if the entire area of the state is covered in solar panels 
    """
    A_panel = 1.5 # m2
    n_panels = A_spacing / A_panel 
    farm_potential = np.multiply(single_panel_potential,n_panels)
    return farm_potential

    

def calc_num_panels(p_land, num_states, state_energy, state_areas):
    # calc energy available if use x percent land of top y most producing states 
    state_energy_sort = state_energy.sort_values(ascending=False)
    state_energy_lim = state_energy_sort[0:num_states]

    total_energy = state_energy_lim.apply(lambda x: x*p_land).sum()

    # calc amount of land used to produce this energy 
    land_used = state_areas.iloc[state_energy_lim.index]["area (m2)"].apply(lambda x: x*p_land).sum()

    # calc number of solar panels on this land, assuming standard panels 
    A_panel = 1.5 # m2
    n_panels = land_used/A_panel 

    return {
        "total_energy (mwh)": total_energy,
        "land_used (m2)": land_used,
        "n_panels": n_panels
    }


In [28]:
power_potential_panel  = calculate_power_panel(F_cur=avg_vals["solar_flux"], T_a=avg_vals["temperature"], w=avg_vals["wind_speed"])

In [37]:
# determine the areas of each state, and determine the power potential if covered with identical arrays of solar panels..

power_potential_panel_farm = calc_power_potential(A_spacing=state_areas["area (m2)"], single_panel_potential=power_potential_panel) # W 

In [41]:
pd.set_option('display.float_format', lambda x: f'{x:,.0f}')

In [105]:
# convert power potentials to energy for a year 
w_to_mw = 1/1e6
w_to_kw = 1/1e3
hours_per_year = 8760.0
mw_to_mwh = hours_per_year

In [75]:
# convert power potentials to energy for a year 
power_potential_panel_farm_mwh = power_potential_panel_farm*w_to_mw*mw_to_mwh  # MW 
p3f = power_potential_panel_farm_mwh

0    14,664,504
1   216,181,365
dtype: float64

In [76]:
test_1 = calc_num_panels(p_land=0.02, num_states=5, state_energy=p3f, state_areas=state_areas)

{k: print_nice(v) for k,v in test_1.items() }

# some consideration of the maximum power point is needed!

31,857,895
5,931,308,441
3,954,205,627


{'total_energy (mwh)': None, 'land_used (m2)': None, 'n_panels': None}

## calculate wind resource 

In [166]:
def calculate_power_potential_turbine(V_m):
    """
    V_m : mean wind speed # m/s # mean wind speed, will get this from a data source for different areas.
    """

    ## --- atmosphereic values # TODO get real values 
    rho = 1.23 # kg/m^3 # air density, should be a function of turbine height 

    ## ---  turbine details #TODO find realistic value 
    # source: https://www.energy.gov/eere/articles/wind-turbines-bigger-better
    D_t = 127 # m 


    # 6.7, the swept area of a wind turbine 
    A_t = np.pi * D_t**2 / 4

    # 6.11, assuming a Rayleigh distribution of wind speeds, the mean power available in the wind is P_m 
    # P_m = 1/2 * rho * A_t * np.sum(f(v) * v**3 )
    P_m = (6/np.pi) * (0.5) * rho * A_t * V_m**3 # W

    # choose turbines that are rated higher than the power available in the wind 
    # has to be in kilowatts for generalized capacity factor equation 
    P_r = np.ceil(P_m/ 500) * 500
    

    # 6.28, generalized capacity factor equation 
    CF = 0.087 * V_m - ((P_r * w_to_kw)/(D_t**2))

    # 6.24, gross annual energy output and power output 
    H = 8760 # non leap year hours in  a day 
    P_t = P_r * CF 
    E_t = P_t * H 

    # account for losses 
    L_td = 16.1 # tranmission and distribution loss, Table 6.3
    L_d = 1.6 # downtime losses, Section 6.6.6.2 ~ Faultstich et al 2011 
    L_c = 0 # curtailment losses, assume storage is available 
    L_a = 0.2 # array losses, # Figure 6.25b, Jacobson and Archer TODO model array losses as function of density... 
    pL_td = 1 - L_td/100 
    pL_d = 1 - L_td/100 
    pL_c = 1 - L_c/100
    pL_a = 1 - L_a/100 
    L_t = (1 - (pL_td * pL_d * pL_c * pL_a )) # compound losses 

    P_t_after_losses = P_t * (1 - L_t)


    return P_m, P_t, CF, P_t_after_losses, P_r

def calc_wind_power_potential(A_spacing,  power_potential_turbine):
    """
    given the area of a state, and the power potential of a panel in that state, calculate the power potential of the state if the entire area of the state is covered in wind turbines, at a comprable density  
    """
    rho_i_compare = 6.64 # W/m2 Table 6.4, mean output power density of onshore europe turbines # TODO see Enevoldsen and Jacobson 2020 to see how array losses compared with density for L_a above 
    n =  (rho_i_compare/ power_potential_turbine) * A_spacing 
    P_t_farm = n * power_potential_turbine

    # TODO can honestly skip this step, given that it will just be a constant function of the output from the previous function, redo solar in this way ...

    return P_t_farm

def calc_num_turbines(p_land, num_states, state_areas, P_t_after_losses, P_r):

    # want to use land in states that are most productive
    state_energy_sort = P_t_after_losses.sort_values(ascending=False)
    state_energy_lim = state_energy_sort[0:num_states]

    # land available to use is a percent of the total land in the state 
    land_avail = state_areas.iloc[state_energy_lim.index]["area (m2)"].apply(lambda x: x*p_land)

    # determine the number of turbines that can fit on land using rated power 
    installed_power_density = 19.8 # mw/km2 => w/m2, # W/m2 Table 6.4, mean output power density of onshore europe turbines

    n_turbines = np.round(installed_power_density / P_r * land_avail )

    installed_power = n_turbines * P_r 

    true_power = n_turbines * P_t_after_losses

    total_energy = true_power.sum()*w_to_mw*mw_to_mwh


    return {
        "n_turbines": n_turbines.dropna(),
        "installed_power": installed_power.dropna(),
        "true_power": true_power.dropna(),
        "capacity_factors": true_power.dropna()/installed_power.dropna(),
        "total_energy (mwh)": total_energy,
        
    }
# mw_to_w = 1e6 
# km2_to_m2 = 1e6 

In [153]:
P_m, P_t, CF, P_t_after_losses, P_r   = calculate_power_potential_turbine(V_m=avg_vals["wind_speed"])

In [160]:
test_1["n_turbines"].dropna()

7    9,667.000
20   2,392.000
33   3,426.000
35   4,970.000
36   3,587.000
dtype: float64

In [170]:
test_1 = calc_num_turbines(p_land=0.02, num_states=2, state_areas=state_areas, P_t_after_losses=P_t_after_losses, P_r=P_r)
print_nice(test_1["total_energy"])
test_1


43,276,424


{'n_turbines': 20   2,392.000
 33   3,426.000
 dtype: float64,
 'installed_power': 20    9,485,476,000.000
 33   12,775,554,000.000
 dtype: float64,
 'true_power': 20   2,092,473,485.803
 33   2,847,757,612.994
 dtype: float64,
 'capacity_factors': 20   0.221
 33   0.223
 dtype: float64,
 'total_energy': 43276424.42545776}

161,456,597


In [138]:
state_wind_power = calc_wind_power_potential(state_areas["area (m2)"], power_potential_turbine_after_losses)

In [146]:
state_wind_energy = state_wind_power*w_to_mw*mw_to_mwh

In [150]:


# {k: print_nice(v) for k,v in test_1.items() }

{'total_energy (mwh)': 7    260,300,192.807
 26   259,110,478.965
 34   210,328,522.140
 4    174,179,672.081
 35   161,225,503.826
 dtype: float64,
 'land_used (m2)': 7    1,449,494,335.710
 26   1,442,869,356.080
 34   1,171,224,647.174
 4      969,928,010.251
 35     897,792,091.693
 Name: area (m2), dtype: float64,
 'n_turbines': 7    0.009
 26   0.009
 34   0.009
 4    0.009
 35   0.009
 dtype: float64,
 'energy_density': 7    0.180
 26   0.180
 34   0.180
 4    0.180
 35   0.180
 dtype: float64}

In [136]:
state_wind_power*w_to_mw #*mw_to_mwh 

0       99,607.088
1      777,462.248
2      138,103.871
3       98,562.634
4      994,176.211
5      195,701.120
6      644,975.684
7    1,485,731.694
8      429,268.661
9      350,816.421
10     129,897.866
11     400,888.814
12     118,351.334
13     156,532.344
14     151,456.974
15     357,478.944
16     104,046.954
17     475,631.782
18     905,009.791
19     416,363.247
20     491,137.434
21     754,154.429
22     592,980.520
23     693,687.547
24      75,879.675
25     548,385.221
26   1,478,941.090
27     344,618.811
28     310,210.119
29     177,208.112
30     570,409.097
31     543,040.548
32     208,870.479
33     661,280.601
34   1,200,505.263
35     920,236.894
36     688,020.163
dtype: float64

In [127]:
power_potential_turbine_after_losses

0    225,153.117
1    345,190.888
2    156,049.028
3    174,251.982
4    404,306.157
5    146,561.442
6    373,315.902
7    676,432.877
8    158,920.752
9    202,354.380
10   275,316.906
11   222,053.465
12   338,978.063
13   193,968.698
14   130,647.527
15   653,062.349
16   169,316.693
17   580,484.060
18   581,666.294
19   557,154.800
20   874,779.885
21   515,182.595
22   181,358.715
23   328,146.122
24   235,251.607
25   150,350.856
26   265,422.917
27   227,086.532
28   180,006.853
29   185,686.183
30   310,362.538
31   646,740.490
32   146,948.323
33   831,219.385
34   215,945.116
35   802,047.977
36   826,772.300
Name: value, dtype: float64

In [118]:
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')
CF # all < 0.45  # C_p = 0.45 # section 6.6.5


0    0.296
1    0.314
2    0.279
3    0.284
4    0.319
5    0.275
6    0.316
7    0.324
8    0.279
9    0.291
10   0.305
11   0.295
12   0.313
13   0.289
14   0.270
15   0.325
16   0.283
17   0.325
18   0.325
19   0.325
20   0.314
21   0.324
22   0.286
23   0.312
24   0.298
25   0.277
26   0.303
27   0.296
28   0.285
29   0.287
30   0.310
31   0.325
32   0.276
33   0.317
34   0.294
35   0.319
36   0.318
Name: value, dtype: float64

In [112]:
power_in_wind

0    1,082,438
1    1,566,351
2      797,166
3      873,425
4    1,805,229
5      757,021
6    1,679,660
7    2,968,946
8      809,367
9      989,249
10   1,285,069
11   1,069,616
12   1,541,320
13     954,945
14     689,343
15   2,862,350
16     852,801
17   2,541,172
18   2,546,427
19   2,440,664
20   3,965,391
21   2,262,514
22     902,850
23   1,497,602
24   1,123,469
25     773,116
26   1,245,417
27   1,090,036
28     897,054
29     920,528
30   1,426,061
31   2,833,781
32     758,955
33   3,728,551
34   1,044,940
35   3,576,701
36   3,705,086
Name: value, dtype: float64

In [113]:
power_potential_turbine

0      320,497
1      491,366
2      222,130
3      248,041
4      575,514
5      208,624
6      531,401
7      962,876
8      226,217
9      288,044
10     391,903
11     316,084
12     482,522
13     276,107
14     185,972
15     929,609
16     241,016
17     826,296
18     827,979
19     793,088
20   1,245,215
21     733,342
22     258,157
23     467,103
24     334,871
25     214,019
26     377,819
27     323,249
28     256,233
29     264,317
30     441,789
31     920,610
32     209,175
33   1,183,208
34     307,389
35   1,141,684
36   1,176,878
Name: value, dtype: float64

In [90]:
np.ceil(192)

192.0

In [91]:
np.ceil(power_potential_turbine * w_to_mw * 1000 / 500) * 500

0    1,500
1    2,000
2    1,000
3    1,000
4    2,000
5    1,000
6    2,000
7    3,000
8    1,000
9    1,000
10   1,500
11   1,500
12   2,000
13   1,000
14   1,000
15   3,000
16   1,000
17   3,000
18   3,000
19   2,500
20   4,000
21   2,500
22   1,000
23   1,500
24   1,500
25   1,000
26   1,500
27   1,500
28   1,000
29   1,000
30   1,500
31   3,000
32   1,000
33   4,000
34   1,500
35   4,000
36   4,000
Name: value, dtype: float64